In [35]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import os
import pickle
from tqdm import tqdm

import grid2op
from lightsim2grid import LightSimBackend

import matplotlib.pyplot as plt

%matplotlib qt



def loss(y_true,y_pred):
        actions, rewards = tf.split(y_true,[3,1],axis=1)        
        policy_gradient = tf.reduce_mean(rewards * actions * tf.math.log(y_pred))
        return -policy_gradient
    
    
def build_n():
    """DQN"""

    input_shape = (1,)

    model = keras.Sequential()
    model.add( keras.layers.BatchNormalization(input_shape=input_shape) )
    model.add( keras.layers.Dense(5, activation='relu') )
    model.add( keras.layers.Dense(5, activation='relu') )
    model.add( keras.layers.Dense(5, activation='relu') )
    model.add( keras.layers.Dense(3, activation='softmax') )
    


    # model.compile(loss=loss)
    def loss(y_true,y_pred):
        actions, rewards = tf.split(y_true,[3,1],axis=1)        
        policy_gradient = tf.reduce_mean(rewards * actions * tf.math.log(y_pred))
        return -policy_gradient
    
    model.compile('rmsprop',loss=loss)
    
    return model


def choose_action(probs):
    """Actor selects next action"""
    l=[]
    for p in probs:
        l.append(np.random.multinomial(1,p))
        
    return np.array(l)

def rew(i):
    if i%3==1:
        return 3
    elif i%3==0:
        return -1
    elif i%3==2:
        return -2
    else:
        raise Error
        
rew = np.vectorize(rew)

def main():
    """DQNs"""
    global new, rewards, y, net
    
    net1 = build_n()
    net2 = build_n()
    fig,ax = plt.subplots(2)

    while True:
        y_1 = np.random.rand(1,1)
        y_2 = np.random.rand(1,1)
        
        probs_1 = net1.predict(y_1)
        
        probs_2 = net2.predict(y_2)
        
        ax[0].clear()
        ax[0].bar(x=[0,1,2],height=probs_1[0])
        ax[0].set_ylim(0,1)
        
        ax[1].clear()
        ax[1].bar(x=[0,1,2],height=probs_2[0])
        ax[1].set_ylim(0,1)

        fig.canvas.draw()
        fig.canvas.flush_events()
                
        act_1=choose_action(probs_1)
        act_2 = choose_action(probs_2)
        rewards_1 = rew(act_1.argmax(1)-act_2.argmax(1))
        rewards_2 = rew(act_2.argmax(1)-act_1.argmax(1))

        net1.fit(y_1,np.array(np.concatenate([act_1,rewards_1.reshape((-1,1))],-1),dtype='float32'),epochs=1,verbose=0)
        net2.fit(y_2,np.array(np.concatenate([act_2,rewards_2.reshape((-1,1))],-1),dtype='float32'),epochs=1,verbose=0)

In [36]:
plt.close()
main()

KeyboardInterrupt: 